# Agenda, day 2

1. Recap
2. Q&A
3. Magic methods
4. Class attributes
5. Finding attributes with ICPO
6. Inheritance -- and how ICPO influences it
7. Three models for method inheritance
8. Data inheritance
9. AMA / where to from here?